### Updates `openalex.works.work_authorships` in Walden End to End workflow

### Step 1: Create enriched authors

In [ ]:
REFRESH MATERIALIZED VIEW openalex.works.work_author_affiliations_mv

In [0]:
DECLARE OR REPLACE VARIABLE max_updated_date TIMESTAMP DEFAULT to_timestamp('1900-01-01');
SET VARIABLE max_updated_date = COALESCE((SELECT MAX(updated_datetime) FROM identifier('openalex' || :env_suffix || '.works.work_authorships')), to_timestamp('1900-01-01'));
-- SET VARIABLE max_updated_date = to_timestamp('2025-12-20');
SELECT max_updated_date;

In [0]:
%run ../utils/variables

In [0]:
CREATE OR REPLACE TABLE identifier('openalex' || :env_suffix || '.works.work_authorships_updates')
CLUSTER BY (work_id) AS (
WITH base_works AS (
    -- Read raw authorships from openalex_works_base (incremental)
    SELECT
        id AS work_id,
        authorships,
        updated_date AS updated_datetime
    FROM identifier('openalex' || :env_suffix || '.works.openalex_works_base')
    WHERE updated_date > max_updated_date
      AND authorships IS NOT NULL
      AND SIZE(authorships) > 0
),
institution_lineage AS (
    SELECT
        institution_id,
        FILTER(lineage_ids, id -> NOT ARRAY_CONTAINS(SUPER_SYSTEM_INSTITUTIONS, id)) AS lineage_ids
    FROM openalex.institutions.institution_ancestors
),
-- 1. Get Institution Details (Grouped by Work/Seq)
author_institutions_with_details AS (
    SELECT
        ai.work_id,
        ai.author_sequence,
        ARRAY_DISTINCT(FLATTEN(COLLECT_SET(ai.raw_countries))) AS raw_parsed_countries,
        COLLECT_SET(
            STRUCT(
                inst.iso3166_code AS country_code,
                inst.display_name,
                CONCAT('https://openalex.org/I', ai.institution_id) AS id,
                ARRAY_SORT(
                    TRANSFORM(
                        ARRAY_COMPACT(CONCAT(ARRAY(ai.institution_id), COALESCE(il.lineage_ids, ARRAY()))), 
                        id -> CONCAT('https://openalex.org/I', id)
                    )
                ) AS lineage,
                CASE 
                    WHEN inst.ror_id IS NULL THEN NULL
                    WHEN inst.ror_id LIKE 'https://ror.org/%' THEN inst.ror_id
                    ELSE CONCAT('https://ror.org/', inst.ror_id) 
                END AS ror,
                inst.type
            )
        ) AS institutions
    FROM (
        SELECT
            work_id,
            author_sequence,
            raw_countries,
            EXPLODE(institution_ids) AS institution_id
        FROM identifier('openalex' || :env_suffix || '.works.work_author_affiliations_mv')
        WHERE institution_ids IS NOT NULL AND SIZE(institution_ids) > 0
    ) ai
    LEFT JOIN openalex.institutions.institutions inst ON inst.id = ai.institution_id
    LEFT JOIN institution_lineage il ON il.institution_id = ai.institution_id
    GROUP BY ai.work_id, ai.author_sequence
),
-- 2. Get Author IDs (Grouped by Work/Seq)
author_id_lookup AS (
    SELECT 
        work_id, 
        author_sequence, 
        MAX(author_id) as author_id
    FROM identifier('openalex' || :env_suffix || '.works.work_author_affiliations_mv')
    GROUP BY work_id, author_sequence
),
-- 3. Enrich Author IDs with Profile Data (OpenAlex Authors + Registry)
author_data_enriched AS (
    SELECT 
        ail.work_id,
        ail.author_sequence,
        ail.author_id,
        -- Priority 1: Existing OpenAlex Author
        -- Priority 2: Newly Minted Registry Author
        COALESCE(oa.display_name, ar.display_name) as best_display_name,
        oa.orcid as best_orcid
    FROM author_id_lookup ail
    -- Join to Main Table (Existing Authors)
    LEFT JOIN openalex.authors.openalex_authors oa 
        ON ail.author_id = oa.id
    -- Join to Registry (New Authors)
    LEFT JOIN openalex.authors.author_registry ar 
        ON ail.author_id = ar.id
),
affiliations_map_ids AS (
    SELECT
        work_id,
        author_sequence,
        raw_affiliation_string,
        TRANSFORM(
            COALESCE(institution_ids, ARRAY()),
            id -> CONCAT('https://openalex.org/I', id)
        ) AS institution_ids
    FROM identifier('openalex' || :env_suffix || '.works.work_author_affiliations_mv')
    WHERE raw_affiliation_string IS NOT NULL
),
affiliations_map AS (
    SELECT
        work_id,
        author_sequence,
        MAP_FROM_ENTRIES(
            ARRAY_DISTINCT(
                COLLECT_LIST(NAMED_STRUCT('key', raw_affiliation_string, 'value', institution_ids))
            )
        ) AS aff_map
    FROM affiliations_map_ids
    GROUP BY work_id, author_sequence
),
-- 4. Build Final Lookup Map
author_institution_lookup AS (
    SELECT
        ade.work_id,
        MAP_FROM_ENTRIES(
            COLLECT_LIST(
                STRUCT(
                    ade.author_sequence,
                    STRUCT(
                        -- Enriched Author Data
                        ade.author_id,
                        ade.best_display_name,
                        ade.best_orcid,
                        
                        -- Institution Data
                        details.institutions,
                        details.raw_parsed_countries,
                        am.aff_map
                    )
                )
            )
        ) AS author_lookup
    FROM author_data_enriched ade
    LEFT JOIN author_institutions_with_details details
        ON ade.work_id = details.work_id 
        AND ade.author_sequence = details.author_sequence
    LEFT JOIN affiliations_map am 
        ON ade.work_id = am.work_id 
        AND ade.author_sequence = am.author_sequence
    GROUP BY ade.work_id
),
exploded_for_parsed_names AS (
    SELECT
        work_id,
        updated_datetime,
        POSEXPLODE(authorships) AS (author_idx, authorship)
    FROM base_works
),
with_parsed_names AS (
    SELECT
        e.work_id,
        e.updated_datetime,
        e.author_idx,
        e.authorship,
        pn.parsed_name
    FROM exploded_for_parsed_names e
    LEFT JOIN identifier('openalex' || :env_suffix || '.authors.parsed_names_lookup') pn
        ON TRIM(e.authorship.raw_author_name) = pn.raw_author_name
),
authorships_with_parsed_names AS (
    SELECT
        work_id,
        updated_datetime,
        TRANSFORM(
            ARRAY_SORT(
                COLLECT_LIST(
                    STRUCT(
                        author_idx,
                        STRUCT(
                            authorship.affiliations AS affiliations,
                            authorship.author AS author,
                            authorship.author_position AS author_position,
                            authorship.author_order_number AS author_order_number,
                            authorship.countries AS countries,
                            authorship.institutions AS institutions,
                            authorship.is_corresponding AS is_corresponding,
                            authorship.raw_affiliation_strings AS raw_affiliation_strings,
                            authorship.raw_author_name AS raw_author_name,
                            parsed_name AS parsed_name
                        ) AS authorship
                    )
                ),
                (left, right) -> CASE
                    WHEN left.author_idx < right.author_idx THEN -1
                    WHEN left.author_idx > right.author_idx THEN 1
                    ELSE 0
                END
            ),
            x -> x.authorship
        ) AS authorships
    FROM with_parsed_names
    GROUP BY work_id, updated_datetime
),
-- 5. Build enriched authorships array
enriched_authorships AS (
    SELECT
        ba.work_id,
        ba.updated_datetime,
        TRANSFORM(
            ba.authorships,
            (auth, idx) -> STRUCT(
                TRANSFORM(
                    COALESCE(auth.raw_affiliation_strings, ARRAY()),
                    s -> STRUCT(
                        COALESCE(ELEMENT_AT(ELEMENT_AT(ail.author_lookup, idx).aff_map, s), ARRAY()) AS institution_ids,
                        s AS raw_affiliation_string
                    )
                ) AS affiliations,
                
                STRUCT(
                    CASE 
                       WHEN ELEMENT_AT(ail.author_lookup, idx).author_id IS NOT NULL 
                       THEN CONCAT('https://openalex.org/A', CAST(ELEMENT_AT(ail.author_lookup, idx).author_id AS STRING))
                       ELSE auth.author.id 
                    END as id,
                    -- Use Display Name from OA/Registry, fallback to raw work data
                    COALESCE(
                        ELEMENT_AT(ail.author_lookup, idx).best_display_name, 
                        auth.author.display_name
                    ) as display_name,
                    -- Use ORCID from OA, fallback to raw work data
                    CASE 
                        WHEN ELEMENT_AT(ail.author_lookup, idx).author_id IS NOT NULL 
                        THEN ELEMENT_AT(ail.author_lookup, idx).best_orcid
                        ELSE auth.author.orcid
                    END as orcid
                ) as author,
                
                auth.author_position,
                auth.author_order_number,
                CASE
                    WHEN ELEMENT_AT(ail.author_lookup, idx).institutions IS NOT NULL 
                         AND SIZE(FILTER(ELEMENT_AT(ail.author_lookup, idx).institutions.country_code, c -> c IS NOT NULL AND c <> '')) > 0
                        THEN ARRAY_SORT(ARRAY_DISTINCT(FILTER(ELEMENT_AT(ail.author_lookup, idx).institutions.country_code, c -> c IS NOT NULL AND c <> '')))
                    WHEN ELEMENT_AT(ail.author_lookup, idx).raw_parsed_countries IS NOT NULL
                        THEN ELEMENT_AT(ail.author_lookup, idx).raw_parsed_countries
                    ELSE ARRAY()
                END AS countries,
                COALESCE(ELEMENT_AT(ail.author_lookup, idx).institutions, ARRAY()) AS institutions,
                auth.is_corresponding,
                auth.raw_affiliation_strings,
                auth.raw_author_name,
                auth.parsed_name
            )
        ) AS authorships
    FROM authorships_with_parsed_names ba
    LEFT JOIN author_institution_lookup ail ON ba.work_id = ail.work_id
)
-- 6. Final select with authors_count, corresponding_author_ids, corresponding_institution_ids
SELECT
    work_id,
    updated_datetime,
    authorships,
    SIZE(authorships) AS authors_count,
    ARRAY_DISTINCT(
        ARRAY_COMPACT(
            TRANSFORM(
                FILTER(authorships, a -> a.is_corresponding = true),
                a -> a.author.id
            )
        )
    ) AS corresponding_author_ids,
    ARRAY_DISTINCT(
        ARRAY_COMPACT(
            FLATTEN(
                TRANSFORM(
                    FILTER(authorships, a -> a.is_corresponding = true),
                    a -> TRANSFORM(a.institutions, i -> i.id)
                )
            )
        )
    ) AS corresponding_institution_ids
FROM enriched_authorships);

### Step 2: Merge enriched updates into final `work_authorships` table

In [0]:
MERGE INTO identifier('openalex' || :env_suffix || '.works.work_authorships') AS target
USING identifier('openalex' || :env_suffix || '.works.work_authorships_updates') AS source
ON target.work_id = source.work_id
WHEN MATCHED THEN UPDATE SET
  target.authorships = source.authorships,
  target.authors_count = source.authors_count,
  target.corresponding_author_ids = source.corresponding_author_ids,
  target.corresponding_institution_ids = source.corresponding_institution_ids,
  target.updated_datetime = source.updated_datetime
WHEN NOT MATCHED THEN INSERT (work_id, authorships, authors_count, corresponding_author_ids, corresponding_institution_ids, updated_datetime)
VALUES (source.work_id, source.authorships, source.authors_count, source.corresponding_author_ids, source.corresponding_institution_ids, source.updated_datetime);